In [ ]:
import glob
import os
import numpy as np
from pcmdi_metrics.graphics import download_archived_results
from pcmdi_metrics.graphics import normalize_by_median
from pcmdi_metrics.graphics import TaylorDiagram
from matplotlib import pyplot as plt
from pcmdi_metrics.graphics import Metrics

In [ ]:
# json_list = sorted(glob.glob(os.path.join(json_dir, '*' + mip + '*' + data_version + '.json')))
json_list1 = sorted(glob.glob("/global/cfs/cdirs/m4581/lee1043/work/cdat/pmp/ACE_evaluation/mean_climate/output/cmip6/amip/v20251020/*.json"))
for json_file in json_list1:
    print(json_file.split('/')[-1])

In [ ]:
json_list2 = sorted(glob.glob("/global/cfs/cdirs/m4581/lee1043/work/cdat/pmp/ACE_evaluation/mean_climate/output/cmip6/amip/v20250915/*.json"))
for json_file in json_list2:
    print(json_file.split('/')[-1])

In [ ]:
json_list = json_list1[:]+json_list2[1:2]

In [ ]:
json_list

# ACE and NGCM-evap

In [ ]:
json_dir_test = './demo_output/climatology-ACE/'
path = "pr_2.5x2.5_regrid2_metrics_climatology-ACE.json"

In [ ]:
json_list_test = sorted(glob.glob(os.path.join(json_dir_test, '*.json')))
for json_file in json_list_test:
    print(json_file.split('/')[-1])

In [ ]:
library = Metrics(json_list)

In [ ]:
json_list_test

In [ ]:
json_list_test_NGCM = sorted(glob.glob(os.path.join('./demo_output/climatology-NGCM-evap/', '*.json')))
for json_file in json_list_test_NGCM:
    print(json_file.split('/')[-1])

In [ ]:
test_case1 = Metrics(json_list_test_NGCM[:])
test_case2 = Metrics(json_list_test[:])
combined = test_case1.merge(test_case2)

In [ ]:
combined.stats

In [ ]:
df_dict = test_case2.df_dict
var_list = test_case2.var_list
var_unit_list = test_case2.var_unit_list
var_ref_dict = test_case2.var_ref_dict
regions = test_case2.regions
stats = test_case2.stats

In [ ]:
print('var_list:', sorted(var_list))
print('var_unit_list:', sorted(var_unit_list))
print('var_ref_dict:', var_ref_dict)

In [ ]:
data_djf = df_dict['rms_xy']['djf']['global'][var_list].to_numpy()
data_mam = df_dict['rms_xy']['mam']['global'][var_list].to_numpy()
data_jja = df_dict['rms_xy']['jja']['global'][var_list].to_numpy()
data_son = df_dict['rms_xy']['son']['global'][var_list].to_numpy()
model_names = df_dict['rms_xyt']['ann']['global']['model'].tolist()
data_all = np.stack([data_djf, data_mam, data_jja, data_son])
print('data.shape:', data_all.shape)
print('len(var_list): ', len(var_list))
print('len(model_names): ', len(model_names))

xaxis_labels = var_list
yaxis_labels = model_names

In [ ]:
data_djf_nor = normalize_by_median(data_djf)
data_mam_nor = normalize_by_median(data_mam)
data_jja_nor = normalize_by_median(data_jja)
data_son_nor = normalize_by_median(data_son)

In [ ]:
data_all_nor = np.stack([data_djf_nor, data_mam_nor, data_jja_nor, data_son_nor])
data_all_nor.shape

In [ ]:
combined_all = library.merge(combined)
# combined_all = library.merge(test_case2)

In [ ]:
combined_all.df_dict['bias_xy']

In [ ]:
def plot_taylor(var, season, region='global', savefig=True):
    fig = plt.figure(figsize=(8, 8))
    stddev = combined_all.df_dict["std_xy"][season][region][var].to_numpy()
    refstd = combined_all.df_dict['std-obs_xy'][season][region][var][0]
    corrcoeff = combined_all.df_dict["cor_xy"][season][region][var].to_numpy()
    models = combined_all.df_dict["cor_xy"][season][region]['model'].to_list()
    var_ref_dict = combined_all.var_ref_dict
    # Customize plot to highlight user's model in red, others in grey
    colors = list()
    labels = list()
    markers = list()
    markersizes = list()
    zorders = list()
    for i, model in enumerate(models):
        if i == len(models)-1:
            labels.append(model)
            colors.append('red')
            markers.append('o')
            markersizes.append(12)
            zorders.append(100)
        elif i == len(models)-2:
            labels.append(model)
            colors.append('blue')
            markers.append('o')
            markersizes.append(12)
            zorders.append(100)
        else:
            if i == 0:
                labels.append('CMIP6 models')
            else:
                labels.append(None)
            colors.append('grey')
            markersizes.append(8)
            markers.append('o')
            zorders.append(10)
    
    fig, ax = TaylorDiagram(stddev, corrcoeff, refstd, fig, colors=colors, normalize=True,
                            labels=labels, markers=markers,
                            markersizes=markersizes, zorders=zorders, ref_label='Ref: '+var_ref_dict[var])
    
    ax.legend(bbox_to_anchor=(1.05, 1.05), loc='upper right', ncol=1)
    fig.suptitle(', '.join([var, season, region]), fontsize=20)
    plt.savefig(f"Figs/Taylor_{var}_{season}_{region}.png", dpi=150, bbox_inches='tight')
    plt.show()

In [ ]:
plot_taylor(var='pr', season='djf', region='global', savefig=True)
plot_taylor(var='pr', season='mam', region='global', savefig=True)
plot_taylor(var='pr', season='jja', region='global', savefig=True)
plot_taylor(var='pr', season='son', region='global', savefig=True)

plot_taylor(var='pr', season='djf', region='NHEX', savefig=True)
plot_taylor(var='pr', season='mam', region='NHEX', savefig=True)
plot_taylor(var='pr', season='jja', region='NHEX', savefig=True)
plot_taylor(var='pr', season='son', region='NHEX', savefig=True)

plot_taylor(var='pr', season='djf', region='SHEX', savefig=True)
plot_taylor(var='pr', season='mam', region='SHEX', savefig=True)
plot_taylor(var='pr', season='jja', region='SHEX', savefig=True)
plot_taylor(var='pr', season='son', region='SHEX', savefig=True)

plot_taylor(var='pr', season='djf', region='TROPICS', savefig=True)
plot_taylor(var='pr', season='mam', region='TROPICS', savefig=True)
plot_taylor(var='pr', season='jja', region='TROPICS', savefig=True)
plot_taylor(var='pr', season='son', region='TROPICS', savefig=True)

# Temperature and psl

In [ ]:
json_list_test_NGCM = sorted(glob.glob(os.path.join('./demo_output/climatology-NGCM/', '*.json')))
for json_file in json_list_test_NGCM:
    print(json_file.split('/')[-1])

In [ ]:
library = Metrics(json_list)
test_case1 = Metrics(json_list_test_NGCM[:])
test_case2 = Metrics(json_list_test[:])
combined = test_case1.merge(test_case2)
combined_all = library.merge(combined)

In [ ]:
# change NGCM name:
for metrics in ['std_xy', 'std-obs_xy', 'cor_xy']:
    for season in ['djf', 'mam', 'jja', 'son']:
        df = combined_all.df_dict[metrics][season]['global']
        df.loc[52, 'model'] = 'NeuralGCM'
# combined_all.df_dict['std_xy']['djf']['global']['model'][52] = 'NeuralGCM'

In [ ]:
plot_taylor(var='ua-200', season='djf', region='global')
plot_taylor(var='ua-200', season='mam', region='global')
plot_taylor(var='ua-200', season='jja', region='global')
plot_taylor(var='ua-200', season='son', region='global')

In [ ]:
plot_taylor(var='ua-850', season='djf', region='global')
plot_taylor(var='ua-850', season='mam', region='global')
plot_taylor(var='ua-850', season='jja', region='global')
plot_taylor(var='ua-850', season='son', region='global')

In [ ]:
plot_taylor(var='va-850', season='djf', region='global')
plot_taylor(var='va-850', season='mam', region='global')
plot_taylor(var='va-850', season='jja', region='global')
plot_taylor(var='va-850', season='son', region='global')

In [ ]:
plot_taylor(var='va-200', season='djf', region='global')
plot_taylor(var='va-200', season='mam', region='global')
plot_taylor(var='va-200', season='jja', region='global')
plot_taylor(var='va-200', season='son', region='global')

In [ ]:
plot_taylor(var='ta-850', season='djf', region='global')
plot_taylor(var='ta-850', season='mam', region='global')
plot_taylor(var='ta-850', season='jja', region='global')
plot_taylor(var='ta-850', season='son', region='global')

In [ ]:
plot_taylor(var='ta-200', season='djf', region='global')
plot_taylor(var='ta-200', season='mam', region='global')
plot_taylor(var='ta-200', season='jja', region='global')
plot_taylor(var='ta-200', season='son', region='global')

In [ ]:
plot_taylor(var='psl', season='djf', region='global')
plot_taylor(var='psl', season='mam', region='global')
plot_taylor(var='psl', season='jja', region='global')
plot_taylor(var='psl', season='son', region='global')

# zg

In [ ]:
library = Metrics(json_list[:])

In [ ]:
json_list_test_NGCM = sorted(glob.glob(os.path.join('./demo_output/climatology-NGCM/', '*.json')))
for json_file in json_list_test_NGCM:
    print(json_file.split('/')[-1])

In [ ]:
test_case1 = Metrics(json_list_test_NGCM[9:10])

In [ ]:
combined_all = library.merge(test_case1)

In [ ]:
print(combined_all.df_dict['bias_xy']['djf']['global'])
# change NGCM name:
for metrics in ['std_xy', 'std-obs_xy', 'cor_xy']:
    for season in ['djf', 'mam', 'jja', 'son']:
        df = combined_all.df_dict[metrics][season]['global']
        df.loc[52, 'model'] = 'NeuralGCM'
# combined_all.df_dict['std_xy']['djf']['global']['model'][52] = 'NeuralGCM'

In [ ]:
fig = plt.figure(figsize=(8, 8))
var = "zg-500"
season = "jja"
region = "global"
stddev = combined_all.df_dict["std_xy"][season][region][var].to_numpy()
refstd = combined_all.df_dict['std-obs_xy'][season][region][var][0]
corrcoeff = combined_all.df_dict["cor_xy"][season][region][var].to_numpy()
models = combined_all.df_dict["cor_xy"][season][region]['model'].to_list()
var_ref_dict = combined_all.var_ref_dict
# Customize plot to highlight user's model in red, others in grey
colors = list()
labels = list()
markers = list()
markersizes = list()
zorders = list()
for i, model in enumerate(models):
    if i == len(models)-1:
        labels.append(model)
        colors.append('red')
        markers.append('o')
        markersizes.append(12)
        zorders.append(100)
    else:
        if i == 0:
            labels.append('CMIP6 models')
        else:
            labels.append(None)
        colors.append('grey')
        markersizes.append(8)
        markers.append('o')
        zorders.append(10)

fig, ax = TaylorDiagram(stddev, corrcoeff, refstd, fig, colors=colors, normalize=True,
                        labels=labels, markers=markers,
                        markersizes=markersizes, zorders=zorders, ref_label='Ref: '+var_ref_dict[var])

ax.legend(bbox_to_anchor=(1.05, 1.05), loc='upper right', ncol=1)
fig.suptitle(', '.join([var, season, region]), fontsize=20)

# Add Watermark
fig.text(0.5, 0.4, 'Example',
        fontsize=100, color='black', alpha=0.1,
        ha='center', va='center', rotation=25)

In [ ]:
def plot_taylor_zg(var, season, region='global', savefig=True):
    fig = plt.figure(figsize=(8, 8))
    stddev = combined_all.df_dict["std_xy"][season][region][var].to_numpy()
    refstd = combined_all.df_dict['std-obs_xy'][season][region][var][0]
    corrcoeff = combined_all.df_dict["cor_xy"][season][region][var].to_numpy()
    models = combined_all.df_dict["cor_xy"][season][region]['model'].to_list()
    var_ref_dict = combined_all.var_ref_dict
    # Customize plot to highlight user's model in red, others in grey
    colors = list()
    labels = list()
    markers = list()
    markersizes = list()
    zorders = list()
    for i, model in enumerate(models):
        if i == len(models)-1:
            labels.append(model)
            colors.append('red')
            markers.append('o')
            markersizes.append(12)
            zorders.append(100)
        else:
            if i == 0:
                labels.append('CMIP6 models')
            else:
                labels.append(None)
            colors.append('grey')
            markersizes.append(8)
            markers.append('o')
            zorders.append(10)
    
    fig, ax = TaylorDiagram(stddev, corrcoeff, refstd, fig, colors=colors, normalize=True,
                            labels=labels, markers=markers,
                            markersizes=markersizes, zorders=zorders, ref_label='Ref: '+var_ref_dict[var])
    
    ax.legend(bbox_to_anchor=(1.05, 1.05), loc='upper right', ncol=1)
    fig.suptitle(', '.join([var, season, region]), fontsize=20)
    plt.savefig(f"Figs/Taylor_{var}_{season}_{region}.png", dpi=150, bbox_inches='tight')
    plt.show()

In [ ]:
plot_taylor_zg(var='zg-500', season='son', region='global', savefig=True)
plot_taylor_zg(var='zg-500', season='jja', region='global', savefig=True)
plot_taylor_zg(var='zg-500', season='djf', region='global', savefig=True)
plot_taylor_zg(var='zg-500', season='mam', region='global', savefig=True)